CSASL (and BCB and ESS, in particular) assembles, integrates, synthesizes, and produces data and information on individual species considered important for our primary mission of informing national conservation policy and resource management decisions. This work involves a number of individual activities that, together, form up one pillar of our overall biogeographic science practice and contribute to the formation of the National Biogeographic Map.

This notebook provides an overview of species data development, management, use, and distribution capabilities. It indicates the current state of play on foundational data management and areas we are working to improve.

In [1]:
import requests
import pysb
from IPython.display import display
from IPython.display import HTML

In [2]:
# In order to access private items in ScienceBase, we need to establish a connection using the pysb package. You will see summary output in this notebook from previous runs of the code when it is posted to GitHub, but we take care not to display particularly sensitive information from the in review items.
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········


In [19]:
# Set up some parameters for this notebook
_gapHabitatMapCollectionItem = "527d0a83e4b0850ea0518326"
_gapSpeciesRangeCollectionItem = "5951527de4b062508e3b1e79"
_gc2BaseURL = "https://gc2.datadistillery.org/api/v1/sql/bcb"

# GAP Species Habitat Models/Maps
We are in the process of releasing the full slate of these products, many of which have been online and used in various ways for some time. Work has been done to refine the models and back-end processing, and all of this will be the subject of an upcoming data review and release. The final model/map products themselves are being assembled in ScienceBase under a [dedicated collection](https://www.sciencebase.gov/catalog/item/527d0a83e4b0850ea0518326) (still private pending review).

In [5]:
# Get the collection item for the GAP Habitat Maps so we can show summary information
gapHabitatMapCollection = sb.get_item(_gapHabitatMapCollectionItem,{'fields':'title,body,purpose,contacts'})

# Display a few summary fields for the habitat map collection
_shortReport = "<h3>"+gapHabitatMapCollection["title"]+"</h3>"
_shortReport = _shortReport+"<h4>Abstract</h4><p>"+gapHabitatMapCollection["body"]+"</p>"
_shortReport = _shortReport+"<h4>Purpose</h4><p>"+gapHabitatMapCollection["purpose"]+"</p>"
_shortReport = _shortReport+"<h4>Contacts</h4>"
for contact in gapHabitatMapCollection["contacts"]:
    _shortReport = _shortReport+"<div>"
    _shortReport = _shortReport+contact["name"]+" ("+contact["type"]+")"
    _shortReport = _shortReport+"</div>"
    
HTML(_shortReport)

In [15]:
# Report on the current number of GAP habitat map items in ScienceBase
habitatMapIDs = sb.get_child_ids(_gapHabitatMapCollectionItem)
_shortReport = "<div>Total number of GAP habitat maps in collection: <strong>"+str(len(habitatMapIDs))+"</strong></div>"

# Get a short list of habitat map items with a search and show a summary
for habitatMapID in habitatMapIDs[0:5]:
    habitatMapItem = sb.get_item(habitatMapID)
    # Display a few summary fields for the habitat map collection
    _shortReport = _shortReport+"<h3>"+habitatMapItem["title"]+"</h3>"
    _shortReport = _shortReport+"<h4>Abstract</h4><p>"+habitatMapItem["body"]+"</p>"
    _shortReport = _shortReport+"<h4>Identifiers</h4>"
    for identifier in habitatMapItem["identifiers"]:
        _shortReport = _shortReport+"<div>"
        _shortReport = _shortReport+identifier["type"]+" ("+identifier["key"]+")"
        _shortReport = _shortReport+"</div>"

HTML(_shortReport)


# GAP Species Ranges
While essentially a part of the GAP species modeling process, the species ranges are their own product with uses beyond the habitat modeling process. Ranges are built against a version of the hydrographic network, slightly modified from the original to meet GAP modeling needs. Range data are also housed in a [dedicated ScienceBase collection](https://www.sciencebase.gov/catalog/item/5951527de4b062508e3b1e79) (still private pending review).

In [18]:
# Get the collection item for the GAP Habitat Maps so we can show summary information
gapSpeciesRangeCollection = sb.get_item(_gapSpeciesRangeCollectionItem,{'fields':'title,body,purpose,contacts,files'})

# Display a few summary fields for the habitat map collection
_shortReport = "<h3>"+gapSpeciesRangeCollection["title"]+"</h3>"
_shortReport = _shortReport+"<h4>Abstract</h4><p>"+gapSpeciesRangeCollection["body"]+"</p>"
_shortReport = _shortReport+"<h4>Purpose</h4><p>"+gapSpeciesRangeCollection["purpose"]+"</p>"
_shortReport = _shortReport+"<h4>Contacts</h4>"
for contact in gapSpeciesRangeCollection["contacts"]:
    _shortReport = _shortReport+"<div>"
    _shortReport = _shortReport+contact["name"]+" ("+contact["type"]+")"
    _shortReport = _shortReport+"</div>"
_shortReport = _shortReport+"<h4>Files</h4>"
for file in gapSpeciesRangeCollection["files"]:
    _shortReport = _shortReport+"<div>"
    _shortReport = _shortReport+file["name"]+" ("+file["title"]+")"
    _shortReport = _shortReport+"</div>"
    
HTML(_shortReport)

## Gap Species Range Data Services
The GAP range data consists of GAP species codes linked to HUC codes (from the derivation of the hydrologic unit database used) and attributes (codes) designating a given hydro unit for origin (native, introduced, reintroduced, vagrant), occurrence (extant, possibly present, potentially present, extirpated), reproductive use (breeding, non-breeding, both) and season (year-round, summer, winter, migratory, vagrant). The data were built in and have been used from a back-end, offline database and are now being released via text files on the collection ScienceBase Item (see above file listing) and an online database instantiation in a system BCB is using called GeoCloud2 (GC2). GC2 provides an API that provides a number of different ways to interact with and use these data in our analytical work.

We are still working on engineering of the GC2 architecture, but the following code block summarizes the current state of these data.

In [21]:
# Get a sample of species range information for a given species
_sampleGapSpeciesCode = "rCOACx"

# Get 20 records of range data by themselves
rangeData = requests.get(_gc2BaseURL+"?q=SELECT * FROM gap.gapspeciesrange WHERE struc = '"+_sampleGapSpeciesCode+"' LIMIT 20").json()
display (rangeData)

# Get the geometry associated with the GAP HUC codes for those same 20 records
rangeGeometry = requests.get(_gc2BaseURL+"?q=SELECT * FROM sfr.sfr_poly WHERE registration->>'sourceLookup' IN (SELECT strhuc12rng FROM gap.gapspeciesrange WHERE struc = '"+_sampleGapSpeciesCode+"' LIMIT 20)").json()
display (rangeGeometry)


{'_execution_time': 0.105,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['gap.gapspeciesrange'],
  'session': None,
  'success': True},
 'features': [{'properties': {'intgaporigin': 1,
    'intgappres': 1,
    'intgaprepro': 3,
    'intgapseas': 1,
    'sourcetable': 'National_GAP_Reptiles_Range_Table',
    'strhuc12rng': '110701060601',
    'struc': 'rCOACx'},
   'type': 'Feature'},
  {'properties': {'intgaporigin': 1,
    'intgappres': 1,
    'intgaprepro': 3,
    'intgapseas': 1,
    'sourcetable': 'National_GAP_Reptiles_Range_Table',
    'strhuc12rng': '110701060602',
    'struc': 'rCOACx'},
   'type': 'Feature'},
  {'properties': {'intgaporigin': 1,
    'intgappres': 1,
    'intgaprepro': 3,
    'intgapseas': 1,
    'sourcetable': 'National_GAP_Reptiles_Range_Table',
    'strhuc12rng': '110701060603',
    'struc': 'rCOACx'},
   'type': 'Feature'},
  {'properties': {'intgaporigin': 1,
    'intgappres': 1,
    'intgaprepro': 3,
    'intgapseas': 1,
    'sourcetable'

{'_execution_time': 12.583,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['sfr.sfr_poly', 'gap.gapspeciesrange'],
  'session': None,
  'success': True},
 'features': [{'geometry': {'coordinates': [[[[-10657716.920477,
        4356680.9484275],
       [-10657582.224149, 4356617.30053],
       [-10657504.293663, 4356625.5782856],
       [-10657261.616598, 4356387.682421],
       [-10657178.126298, 4356227.2424034],
       [-10657005.582078, 4356124.8998431],
       [-10656847.517122, 4356126.3139023],
       [-10656471.256789, 4356156.7478504],
       [-10656298.709347, 4356196.853425],
       [-10656115.030961, 4356253.5939446],
       [-10655966.978415, 4356275.7058222],
       [-10655867.905156, 4356279.7436081],
       [-10655702.029192, 4356257.6579145],
       [-10655598.503937, 4356220.3159174],
       [-10655198.872961, 4356160.8093647],
       [-10655132.074684, 4356166.3240038],
       [-10655049.699335, 4356169.1684992],
       [-10654868.255827, 4356170.48451

## To Dos - Gap Species Ranges
* Decide on individual range item metadata conventions and build range items
* Mint/reserve DOIs for range items to be activated upon release
